In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, LSTM, Dropout, Reshape
from tensorflow.keras.layers import Bidirectional, TimeDistributed
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.models import load_model

%matplotlib inline

In [2]:
X_train, X_valid, X_test = np.load('./X_train.npy'), np.load('./X_valid.npy'), np.load('./X_test.npy')
y_train, y_valid, y_test = np.load('./Y_train.npy'), np.load('./Y_valid.npy'), np.load('./Y_test.npy')
print(X_train.shape, X_valid.shape, X_test.shape)
print(y_train.shape, y_valid.shape, y_test.shape)

(25711, 70, 4) (7093, 70, 4) (4306, 70, 4)
(25711, 70) (7093, 70) (4306, 70)


In [3]:
def matching(preds, true):
    total, match = 0, 0
    for p, t in zip(preds, true):
        if np.argmax(p) in np.nonzero(t)[0]:
            match += 1
        total += 1
    return match, total

In [4]:
K = 3
counts = np.zeros((2*K+1, 4))
for target, seq in zip(y_train, X_train):
    for bp in np.nonzero(target)[0]:
        if 0 > bp-K or bp+K+1 > seq.shape[0]: continue
        counts = counts + seq[bp-K: bp+K+1]
print(counts.T)

[[ 8658.  8037.  8984. 20929.  9468.  9994.  9281.]
 [12046.  8055. 11683.  6820. 11738. 11016. 10508.]
 [ 8100.  7269.  8965.  5728.  8041.  7817.  8519.]
 [10247. 15690.  9419.  5574.  9804. 10224. 10743.]]


In [5]:
class ModelTrainer:
    def __init__(self, model):
        self.model = model
        self.train_auc = []
        self.train_match = []
        self.valid_auc = []
        self.valid_match = []
        
    def train(self, X_train, X_valid,
              y_train, y_valid, PATIENCE = 15, EPOCHS = 30):
        print(self.model.summary())
        for i in range(EPOCHS):
            self.model.fit(X_train, y_train, epochs = 1, verbose = 0, batch_size = 8)
            self._evaluate(X_train, X_valid, y_train, y_valid)
            if (i > PATIENCE
                and max(self.valid_match[-PATIENCE:])
                < max(self.valid_match)):
                break
            print(i, self.valid_match[-1], self.train_match[-1])
        self._plot_scores()
        print(max(self.valid_match), max(self.valid_auc))
    
    def predict(X):
        return self.model.predict(X)
                
    def _evaluate(self, X_train, X_valid, y_train, y_valid):
        valid_preds = self.model.predict(X_valid, verbose=0)
        train_preds = self.model.predict(X_train, verbose=0)
        self.valid_match += [matching(valid_preds, y_valid)[0]
                             / float(y_valid.shape[0])]
        self.train_match += [matching(train_preds, y_train)[0]
                             / float(y_train.shape[0])]
        self.valid_auc += [metrics.roc_auc_score(y_valid.flatten(),
                                                 valid_preds.flatten())]
        self.train_auc += [metrics.roc_auc_score(y_train.flatten(),
                                                 train_preds.flatten())]
    
    def _plot_scores(self):
        plt.plot(self.valid_match, label = 'Validation')
        plt.plot(self.train_match, label = 'Training')
        plt.ylabel('Match')
        plt.xlabel('epoch')
        plt.legend()
        plt.show()
        
        plt.plot(self.valid_auc, label = 'Validation')
        plt.plot(self.train_auc, label = 'Training')
        plt.ylabel('auROC') 
        plt.xlabel('epoch')
        plt.legend()
        plt.show()

In [6]:
new_model = load_model('./2layer.h5')
new_model.summary()

2021-11-16 15:11:50.512913: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-16 15:11:50.887104: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22298 MB memory:  -> device: 0, name: GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 70, 64)           9472      
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 70, 64)           24832     
 nal)                                                            
                                                                 
 time_distributed (TimeDistr  (None, 70, 1)            65        
 ibuted)                                                         
                                                                 
Total params: 34,369
Trainable params: 34,369
Non-trainable params: 0
_________________________________________________________________


In [ ]:
ModelTrainer(model=new_model).train(X_train[:, :, :4], X_valid[:, :, :4],
                          y_train.reshape(-1, 70, 1), y_valid.reshape(-1, 70, 1))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 70, 64)           9472      
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 70, 64)           24832     
 nal)                                                            
                                                                 
 time_distributed (TimeDistr  (None, 70, 1)            65        
 ibuted)                                                         
                                                                 
Total params: 34,369
Trainable params: 34,369
Non-trainable params: 0
_________________________________________________________________
None


2021-11-16 15:12:00.360070: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


0 0.6124347948681799 0.6538057640698534
1 0.6083462568729734 0.6525222667340826
2 0.5777527139433244 0.6417875617440006
